# Summary

* This is a tutorial on using Python for accessing the Tahoe-100 dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#obs-cell-metadata) for a description of the obs metadata.

# Setup

### Installation

If needed, install the necessary dependencies.

You can use the conda environment provided in this git repository. To do so:

In [3]:
!which conda && conda env create -q -f ../arc_py_conda_env.yml

/home/nickyoungblut/miniforge3/condabin/conda
Channels:
 - conda-forge
 - bioconda
 - pytorch
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



python-3.13.2        | 31.7 MB   |                                       |   0% 
scipy-1.15.2         | 16.4 MB   |                                       |   0% 

statsmodels-0.14.4   | 11.7 MB   |                                       |   0% 


scikit-learn-1.6.1   | 10.1 MB   |                                       |   0% 



numpy-2.1.3          | 8.0 MB    |                                       |   0% 




jupyterlab-4.3.5     | 7.3 MB    |                                       |   0% 





babel-2.17.0         | 6.6 MB    |                                       |   0% 






numba-0.61.0         | 5.6 MB    |                                       |   0% 



### Load dependencies

In [ ]:
import os
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs

In [4]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()

### Data location

In [6]:
# GCS bucket path
gcp_base_path = "gs://arc-ctc-tahoe100/2025-02-25/"

# Obs metadata

* `obs` ≃ cell

### Per-sample

* Useful for quickly summarizing the per-sample metadata (a small file versus the entire obs metadata file; see below).

In [8]:
# path to sample metadata
infile = os.path.join(gcp_base_path, 'metadata', 'sample_metadata.parquet.gz')

In [9]:
# read just the first 3 rows
sample_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(3).to_pandas()
sample_metadata

,sample,plate,mean_gene_count,mean_tscp_count,mean_mread_count,mean_pcnt_mito,drug,drugname_drugconc
0,smp_1495,plate1,1354.169768,2027.115940,2444.032416,0.033956,Infigratinib,"[('Infigratinib', 0.05, 'uM')]"
1,smp_1496,plate1,1404.454157,2226.282791,2690.685970,0.071723,Erdafitinib,"[('Erdafitinib ', 0.05, 'uM')]"
2,smp_1497,plate1,1205.267794,1859.375821,2246.200127,0.084853,Everolimus,"[('Everolimus', 0.05, 'uM')]"


In [10]:
# select certain columns and row filtering
columns_to_read = ['sample', 'plate', 'mean_gene_count']  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
sample_metadata = dataset.to_table(filter=(ds.field('mean_gene_count') > 2000), columns=columns_to_read).to_pandas()
sample_metadata 

,sample,plate,mean_gene_count
0,smp_1598,plate2,2196.279615
1,smp_1604,plate2,2039.014042
2,smp_1605,plate2,2043.083782
3,smp_2044,plate6,2155.888048
4,smp_2046,plate6,2072.778266
5,smp_2054,plate6,2035.670581
6,smp_2056,plate6,2058.926286
7,smp_2060,plate6,2094.526584
8,smp_2066,plate6,2229.688468
9,smp_2067,plate6,2192.958014


In [12]:
# get the number of samples
columns_to_read = ["sample"]  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
sample_count = dataset.to_table(columns=columns_to_read).to_pandas()["sample"].nunique()
print(f"Number of samples: {sample_count}")

Number of samples: 1344


In [13]:
# get samples per plate
columns_to_read = ["plate", "sample"]  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
samples_per_plate = dataset.to_table(columns=columns_to_read).to_pandas().groupby("plate").size()
samples_per_plate

plate
plate1     96
plate10    96
plate11    96
plate12    96
plate13    96
plate14    96
plate2     96
plate3     96
plate4     96
plate5     96
plate6     96
plate7     96
plate8     96
plate9     96
dtype: int64

### Per-observation

* `obs` ~= cells
* For the sake of this tutorial, we will just pull the first 100000 observations.

In [15]:
infile = os.path.join(gcp_base_path, 'metadata', 'obs_metadata.parquet.gz')

In [16]:
# read a sample of the metadata
obs_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(100000).to_pandas()
obs_metadata

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
0,smp_2359,1379,2172,2559,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1478,lib_1681,01_001_001,0.029926,-0.229665,-0.190110,G1,full,NCI-H1573,plate10
1,smp_2359,975,1256,1470,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_0459,lib_1681,01_002_149,0.026274,-0.167578,-0.132784,G1,full,NCI-H460,plate10
2,smp_2359,865,1239,1446,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_C466,lib_1681,01_003_052,0.033898,-0.200957,-0.161538,G1,full,hTERT-HPNE,plate10
3,smp_2359,393,484,559,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1724,lib_1681,01_003_090,0.037190,-0.052746,-0.076190,G1,minimal,SW48,plate10
4,smp_2359,2657,5325,6269,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1285,lib_1681,01_003_093,0.017465,-0.636364,-0.614103,G1,full,HOP62,plate10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,smp_2430,1481,2171,2559,"[('γ-Oryzanol', 0.05, 'uM')]",γ-Oryzanol,CVCL_0366,lib_1682,72_141_112,0.042837,0.000000,-0.100386,S,full,SNU-423,plate10
99996,smp_2430,1430,2119,2496,"[('γ-Oryzanol', 0.05, 'uM')]",γ-Oryzanol,CVCL_0371,lib_1682,72_141_131,0.074563,-0.009524,-0.028340,G1,full,KATO III,plate10
99997,smp_2430,827,1044,1201,"[('γ-Oryzanol', 0.05, 'uM')]",γ-Oryzanol,CVCL_1693,lib_1682,72_141_184,0.045019,-0.028571,-0.057324,G1,full,SHP-77,plate10
99998,smp_2430,875,1153,1333,"[('γ-Oryzanol', 0.05, 'uM')]",γ-Oryzanol,CVCL_0320,lib_1682,72_142_157,0.071119,-0.061905,-0.042970,G1,full,HT-29,plate10


In [20]:
# sample count
obs_metadata["sample"].nunique()

96

In [18]:
# gene count distribution
pd.options.display.float_format = '{:.0f}'.format
obs_metadata["gene_count"].describe()

count   100000
mean      1382
std        735
min        268
25%        896
50%       1209
75%       1661
max       9395
Name: gene_count, dtype: float64

In [19]:
# tscp (UMI) count distribution
pd.options.display.float_format = '{:.0f}'.format
obs_metadata["tscp_count"].describe()

count   100000
mean      2214
std       1833
min        392
25%       1230
50%       1748
75%       2583
max      54006
Name: tscp_count, dtype: float64

# Reading in h5ad files

* For this tutorial, we will be reading in a subsampled version of 1 h5ad file, since the per-plate h5ad files are rather large. 

In [29]:
infile = "gs://arc-ctc-tahoe100/2025-02-25/tutorial/plate3_2k-obs.h5ad.gz"

In [32]:
with fs.open(infile, 'rb') as f:
    adata = sc.read_h5ad(f)
adata

AnnData object with n_obs × n_vars = 2000 × 62710
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'id', 'drugname_drugconc', 'drug', 'INT_ID', 'NUM.SNPS', 'NUM.READS', 'demuxlet_call', 'BEST.GUESS', 'BEST.LLK', 'NEXT.GUESS', 'NEXT.LLK', 'DIFF.LLK.BEST.NEXT', 'BEST.POSTERIOR', 'SNG.POSTERIOR', 'cell_line', 'SNG.BEST.LLK', 'SNG.NEXT.GUESS', 'SNG.NEXT.LLK', 'SNG.ONLY.POSTERIOR', 'DBL.BEST.GUESS', 'DBL.BEST.LLK', 'DIFF.LLK.SNG.DBL', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'cell_line_orig', 'pass_filter', 'cell_name'

In [28]:
adata.obs

,sample,species,gene_count,tscp_count,mread_count,bc1_wind,bc2_wind,bc3_wind,bc1_well,bc2_well,...,DIFF.LLK.SNG.DBL,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,cell_line_orig,pass_filter,cell_name
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,,,,,,
01_001_117-lib_1009,smp_1687,hg38,1110,1404,1655,1,1,117,A1,A1,...,-2,lib_1009,01_001_117,0,0,0,G2M,CVCL_1693,full,SHP-77
01_001_122-lib_1009,smp_1687,hg38,1011,1324,1577,1,1,122,A1,A1,...,3,lib_1009,01_001_122,0,-0,-0,G1,CVCL_1495,full,NCI-H1792
01_001_172-lib_1009,smp_1687,hg38,835,1042,1240,1,1,172,A1,A1,...,1,lib_1009,01_001_172,0,-0,0,G2M,CVCL_0399,full,LoVo
01_002_058-lib_1009,smp_1687,hg38,754,902,1040,1,2,58,A1,A2,...,-1,lib_1009,01_002_058,0,-0,-0,G1,CVCL_1056,full,A498
01_002_063-lib_1009,smp_1687,hg38,1546,2288,2695,1,2,63,A1,A2,...,-1,lib_1009,01_002_063,0,0,0,G2M,CVCL_0480,full,PANC-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
04_052_095-lib_1009,smp_1690,hg38,446,561,675,4,52,95,A4,E4,...,1,lib_1009,04_052_095,0,-0,-0,G1,CVCL_1119,minimal,CFPAC-1
04_052_131-lib_1009,smp_1690,hg38,940,1320,1591,4,52,131,A4,E4,...,1,lib_1009,04_052_131,0,-0,-0,G1,CVCL_1119,full,CFPAC-1
04_052_160-lib_1009,smp_1690,hg38,1486,2132,2538,4,52,160,A4,E4,...,-2,lib_1009,04_052_160,0,0,-0,S,CVCL_0359,full,J82


#### Next steps

You can then use the anndata object for various downsteam analyses

# Downloading files

You can use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download any of the files in the bucket
and work with them locally. 

Please be considerate to the cost of egress (downloading data) from the cloud.

For example:

```bash
gsutil cp gs://arc-ctc-tahoe100/2025-02-25/tutorial/plate3_2k-obs.h5ad.gz .
```

***

# sessionInfo

In [1]:
!pip list

Package                   Version
------------------------- --------------
aiohappyeyeballs          2.4.6
aiohttp                   3.11.12
aiosignal                 1.3.2
anndata                   0.11.3
anyio                     4.8.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
array_api_compat          1.10.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
blinker                   1.9.0
Brotli                    1.1.0
cached-property           1.5.2
cachetools                5.5.2
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
cryptography              44.0.1
cycler                    0.12.1
debugpy      